In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration

    return move, pile  # action

In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side

        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")

    return wins['A'], wins['B']

In [4]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random  498    Random  502
1000 games,     Guru 1000    Random    0
1000 games,   Random    4      Guru  996
1000 games,     Guru  950      Guru   50


In [21]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game
                
            elif np.max(qtable[st2[0], st2[1], st2[2]]) >= Reward:
                # immediate loss - penalize it
                qtable_update(-Reward, st1, move, pile, np.min(qtable[st2[0], st2[1], st2[2]]))

            else:
                # not immediate loss - reward it
                qtable_update(Reward, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))                
                
            #qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))

            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [22]:
%%time
nim_qlearn(100000)

Wall time: 5.36 s


In [23]:
# Play games
play_games(1000, 'Qlearner', 'Guru')
play_games(1000, 'Random', 'Qlearner') ;

1000 games, Qlearner  825      Guru  175
1000 games,   Random    8  Qlearner  992


In [8]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]

1000 games, Qlearner  503    Random  497
1000 games, Qlearner  521    Random  479
1000 games, Qlearner  656    Random  344
1000 games, Qlearner  720    Random  280
1000 games, Qlearner  710    Random  290
1000 games, Qlearner  731    Random  269
1000 games, Qlearner  728    Random  272
Wall time: 6.4 s


In [9]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(wins)

[0.503, 0.521, 0.656, 0.72, 0.71, 0.731, 0.728]


In [10]:
# Function to print the entire set of states
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)

        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r

            print(s, file=fout)

qtable_log('qtable_debug.txt')